In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import shutil
import pyperclip
import time
import pytz
import datetime
import os
from tqdm import tqdm
import random

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
    options = webdriver.FirefoxOptions()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
    options.add_argument('E:/Programs/geckodriver.exe')
    # options.add_argument('--headless')
    options.add_argument("--start-with-cache")
    options.add_argument("0")
    options.add_argument("--disk-cache-size=0")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

def save_data_to_disk(data, filename):
    max_attempts = 5  # Maximum number of attempts
    wait_time = 5  # Time to wait between attempts (in seconds)
    
    for _ in range(max_attempts):
        try:
            backup_filename = filename + 'bak.parquet'
            if os.path.isfile(backup_filename):
                os.remove(backup_filename)  # Remove old backup before creating a new one
            shutil.copyfile(filename, backup_filename)
            
            if os.path.isfile(filename):
                existing_df = pd.read_parquet(backup_filename)
                data = pd.concat([existing_df, data], ignore_index=True)
                data.to_parquet(filename, index=False)
            else:
                data.to_parquet(filename, index=False)
            break  # Break out of the loop if successful
        except Exception as e:
            print(f"Error: {e}")
            print(f"File '{filename}' is in use. Waiting {wait_time} seconds before trying again...")
            time.sleep(wait_time)
    else:
        print(f"Failed to save data to '{filename}' after {max_attempts} attempts.")

In [3]:
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [4]:
driver.get(f"https://www.linkedin.com/authwall?trk=qf&original_referer=&sessionRedirect=https%3A%2F%2Fwww.linkedin.com%2F")
Sign_in_button = driver.find_element(By.XPATH, "/html/body/div/main/div/form/p/button").click()
time.sleep(5)
email_phone = driver.find_element(By.XPATH, '//*[@id="session_key"]')
password = driver.find_element(By.XPATH, '//*[@id="session_password"]')
email_phone.send_keys("pannersmail@gmail.com")
# email_phone.send_keys("urvibabla@gmail.com")
# email_phone.send_keys("r.panneerselvam@firebird.net.in")
password.send_keys("MK90473@yahoo.com")
# password.send_keys("SkillIQ@123")

# email_phone.send_keys("chakrabortiarjun@gmail.com")
# password.send_keys("Reafactor_Academy")

# email_phone.send_keys("renjithcm.renju@gmail.com")
# password.send_keys("Renjith#@1")
submit_button = driver.find_element(By.XPATH, "/html/body/div/main/div/div/form[1]/div[2]/button")
submit_button.click()

wait = WebDriverWait(driver, 4)

In [5]:
driver.switch_to.window(driver.window_handles[0])

# connection page scroller.------------------

In [ ]:
#------------------------------------------------------------------------------------------------------
pg = 1
while True:    
    # url = f'https://www.linkedin.com/search/results/people/?geoUrn=[%22102713980%22]&keywords=%22head%22%20%22compensation%22&origin=FACETED_SEARCH&page='+str(pg)
    url = f'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22102713980%22%5D&industry=%5B%223099%22%2C%227%22%5D&keywords=head%20talent%20acquisition&network=%5B%22S%22%2C%22O%22%5D&origin=FACETED_SEARCH&page='+str(pg)
    driver.get(url)
    input()
    pg += 1

# --messaging--from--connections--people scroller--------------------------

In [8]:
(driver.find_elements(By.CSS_SELECTOR,'.scaffold-finite-scroll__content li'))

40

In [ ]:

li_elements = driver.find_elements(By.XPATH, "//div[@class='scaffold-finite-scroll__content']/ul/li")
print(len(li_elements))

In [ ]:
st = 0

In [ ]:
while st <= len(li_elements):
    try:
        li_elements[st].find_element(By.TAG_NAME, "button").click()
        print(f'\rProgress: {st}', end='')
        input()
        close = driver.find_element(By.CSS_SELECTOR, ".msg-overlay-bubble-header__controls.display-flex.align-items-center").find_elements(By.TAG_NAME, "button")
        close[-1].click()
        st+=1
    except IndexError:
        li_elements = driver.find_elements(By.XPATH, "//div[@class='scaffold-finite-scroll__content']/ul/li")

# connection's contact finder.

### find all links.

In [ ]:
linklen = int(driver.find_element(By.CLASS_NAME, "pb2").text.split(' ')[0])
linklen

In [ ]:
links = []

In [ ]:
urlno = 1
while len(links) < linklen-1:    
    url = f'https://www.linkedin.com/search/results/people/?geoUrn=%5B%22102713980%22%5D&keywords=-recruiter&network=%5B%22F%22%5D&origin=GLOBAL_SEARCH_HEADER&page='+str(urlno)+f''
    driver.get(url)
    time.sleep(4)

    li_elements = driver.find_elements(By.CLASS_NAME, "reusable-search__result-container")
    for li in li_elements:
        link_element = li.find_element(By.TAG_NAME, "a")
        link = link_element.get_attribute("href")
        links.append(link)
    urlno +=1
print(len(links))

In [ ]:
len(links)

### drop duplicates and append the newlinks for later referance

In [ ]:
linksdf = pd.read_csv('links.csv')
linksdf.drop_duplicates(inplace=True)
for i in links:
    if i in linksdf['links'].tolist():
        links.remove(i)
# if links != []:    
#     #index=[0]
#     linksdf = pd.concat([linksdf,pd.DataFrame(links)],ignore_index=True)
#     linksdf.to_csv('links.csv',index=False)

In [ ]:
# pd.DataFrame(links).to_csv('links.csv',index=False)

### find contacts.

In [ ]:
df = pd.DataFrame()

In [ ]:
strng = 0

In [ ]:
for linkind in range(strng,len(links_for_contact)):
    driver.get(links_for_contact[linkind])
    time.sleep(5)
    data = {}
    #------------------------------------------------------------------------------------------------------
    data['name'] = driver.find_element(By.CLASS_NAME, "text-heading-xlarge").text
    data['title_text'] = driver.find_element(By.CLASS_NAME, "text-body-medium").text
    data['link'] = links_for_contact[linkind]
    expind = 0
    conts = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.artdeco-card.pv-profile-card.break-words.mt2 h2')))
    for contind in range(len(conts)):
        if 'experience' in conts[contind].text.lower():
            expind = contind
            break 
    liele = driver.find_elements(By.CSS_SELECTOR, '.artdeco-card.pv-profile-card.break-words.mt2')[expind].find_element(By.CSS_SELECTOR,'.pvs-list__outer-container > ul > li')
    data['job_title'] = liele.find_element(By.CSS_SELECTOR, '.display-flex.flex-wrap.align-items-center.full-height').text.split('\n')[0]
    data['company'] = liele.find_element(By.CSS_SELECTOR, '.t-14.t-normal').text.split('\n')[0]
    try:
        data['location'] = liele.find_elements(By.CSS_SELECTOR, '.t-14.t-normal.t-black--light')[1].text.split('\n')
    except:
        None
    try:
        data['duration'] = liele.find_elements(By.CSS_SELECTOR, '.t-14.t-normal.t-black--light')[0].text.split('\n')
    except:
        None
    #------------------------------------------------------------------------------------------------------
    driver.find_element(By.ID, "top-card-text-details-contact-info").click()
    time.sleep(3)
    for i in driver.find_elements(By.CLASS_NAME, "pv-contact-info__contact-type"):
        cdet = i.text.split('\n')
        data[cdet[0]] = cdet[1]
    try:
        temp = pd.DataFrame(data,index=[0])
    except:
        temp = pd.DataFrame(data)
    temp = temp[[i for i in temp.columns if 'rofile' not in i]]
    df = pd.concat([df, temp], ignore_index=True)
    linksdf = pd.concat([linksdf,pd.DataFrame({'links': [links[linkind]]})],ignore_index=True)
    print(f'\rProgress: {linkind}', end='')
    if linkind > 79:
        break
    strng = linkind

In [ ]:
df[df['Phone'].notna()]

In [ ]:
existing_df = pd.read_excel('All_Contacts.xlsx')
existing_df = pd.concat([existing_df,df],ignore_index=True)
existing_df.drop(columns=[col for col in existing_df.columns if 'rofile' in col],inplace=True)
existing_df.drop_duplicates(inplace=True)
existing_df.reset_index(drop=True, inplace=True)
linksdf.drop_duplicates(inplace=True)
linksdf.reset_index(drop=True, inplace=True)
existing_df.to_excel('semiconductor_and_embedded.xlsx',index=False)
linksdf.to_csv('links.csv',index=False)

In [ ]:
existing_df[existing_df['Phone'].notna()]

# END